In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import pingouin as pg
from scipy.stats import shapiro, normaltest
import numpy as np

import warnings
warnings.filterwarnings('ignore')

probe_ord = ['NNXr', 'NNXo']

from acr.plots import lrg
plt.rcdefaults()
lrg()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from acr.utils import *

In [ ]:
save_figs = True
write_stats = True

In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swisin'
SUBJECT_TYPE = 'som'
MAIN_COLOR = SOM_BLUE if SUBJECT_TYPE == 'som' else ACR_BLUE

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
notebook_figure_root = f'{acr.utils.PAPER_FIGURE_ROOT}/firing_rates/tonic_comp'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp, update=False)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=False)

In [ ]:
fr_dfs = {}
for subject, exp in zip(subjects, exps):
    fr = dag.load_raw_fr_df(subject, exp, data_dir=f'{acr.utils.pub_data_root}/raw_fr/by_probe')
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject])
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='condition', ref_to_val='early_sd', col_to_relativize='fr', on=['probe'])
    fr_dfs[subject] = fr
frdf = pl.concat(fr_dfs.values())

In [ ]:
stimf = frdf.filter(pl.col('condition')=='stim')
stim_means = stimf.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
stim_means = stim_means.sort(['subject', 'probe'])
#stim_chan_means = stimf.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=stim_chan_means, x='channel', y='fr_rel', row='subject', hue='probe', hue_order=probe_ord, palette=[NNXR_GRAY, MAIN_COLOR], kind='bar', height=6, aspect=3)

In [ ]:
acr.plots.supl()

In [ ]:
fig_id = 'tonic_som_fr_during_stim'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxo_ton = stim_means.filter(pl.col('probe')=='NNXo')['fr_rel'].to_numpy()
nnxr_ton = stim_means.filter(pl.col('probe')=='NNXr')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr_ton, nnxo_ton, colors=[NNXR_GRAY, MAIN_COLOR], alphas=[0.9, 0.6], fsize=(3.5, 4), one_sided=False)


ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr_ton - nnxo_ton
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr_ton, nnxo_ton, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr_ton, nnxo_ton, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr_ton, 'off_induction': nnxo_ton, 'subject': np.arange(len(nnxr_ton))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'tonic_som_fr_during_stim__NORM'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxo_ton_norm = nnxo_ton/nnxr_ton
nnxr_ton_norm = nnxr_ton/nnxr_ton

f, ax = acr.plots.gen_paired_boxplot(nnxr_ton_norm, nnxo_ton_norm, colors=[NNXR_GRAY, MAIN_COLOR], alphas=[0.9, 0.6], fsize=(3.5, 4), one_sided=True)


ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# MATCHED SWI

In [ ]:
swi_exps = []
for sub in subjects:
    swi_exps.append(acr.utils.sub_swi_exps[sub][0])
#notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/analysis/sub_analysis'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, swi_exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=False)

In [ ]:
fr_dfs = {}
for subject, exp in zip(subjects, swi_exps):
    print(subject)
    fr = dag.load_raw_fr_df(subject, exp, data_dir=f'{acr.utils.pub_data_root}/raw_fr/by_probe')
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject])
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='condition', ref_to_val='early_sd', col_to_relativize='fr', on=['probe'])
    fr_dfs[subject] = fr
frdf_swi = pl.concat(fr_dfs.values())

In [ ]:
stimf_swi = frdf_swi.filter(pl.col('condition')=='stim')
stim_means_swi = stimf_swi.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
stim_means_swi = stim_means_swi.sort(['subject', 'probe'])
#stim_chan_means_swi = stimf_swi.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=stim_chan_means_swi, x='channel', y='fr_rel', row='subject', hue='probe', hue_order=probe_ord, palette=[NNXR_GRAY, MAIN_COLOR], kind='bar', height=6, aspect=3)

In [ ]:
nnxo_opi = stim_means_swi.filter(pl.col('probe')=='NNXo')['fr_rel'].to_numpy()
nnxr_opi = stim_means_swi.filter(pl.col('probe')=='NNXr')['fr_rel'].to_numpy()
nnxr_norm_opi = nnxr_opi/nnxr_opi
nnxo_norm_opi = nnxo_opi/nnxr_opi

In [ ]:
fig_id = 'OFF-IND_som_fr_during_stim'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxr_opi, nnxo_opi, colors=[NNXR_GRAY, MAIN_COLOR], alphas=[0.8, 0.8], fsize=(3.5, 4), one_sided=False)


ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr_opi - nnxo_opi
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr_opi, nnxo_opi, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr_opi, nnxo_opi, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr_opi, 'off_induction': nnxo_opi, 'subject': np.arange(len(nnxr_opi))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'OFF-IND_som_fr_during_stim__NORM'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxr_norm_opi, nnxo_norm_opi, colors=[NNXR_GRAY, MAIN_COLOR], alphas=[0.8, 0.8], fsize=(3.5, 4), one_sided=True)


ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
fig_id = 'ton-vs-off_som_fr_during_stim'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxo_ton_norm, nnxo_norm_opi, colors=[MAIN_COLOR, MAIN_COLOR], alphas=[0.6, 0.9], fsize=(3.5, 4), one_sided=False)


ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Tonic', 'OFF Induction'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxo_norm_opi - nnxo_ton_norm
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxo_norm_opi, nnxo_ton_norm, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxo_norm_opi, nnxo_ton_norm, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxo_norm_opi, 'off_induction': nnxo_ton_norm, 'subject': np.arange(len(nnxo_norm_opi))})
    pu.write_source_data(source_data, stats_name)
stats